In [13]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# model.invoke("Avatar's director, starring, budget, revenue, genre, short summary")

In [14]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
# 감독, 주요 출연진, 예산, 흥행 수익, 장르, 간략한 줄거리
examples = [
    {"input": "Matrix", "output": "{'Director': 'Wachowski', 'Starring': 'Keanu Reeves', 'Budget': '63 million', 'Revenue': '463 million', 'Genre': 'Action, Sci-Fi', 'Summary': 'A computer hacker learns about the true nature of his reality and his role in the war against its controllers.'}"},
    {"input": "Avatar", "output": "{'Director': 'James Cameron', 'Starring': 'Sam Worthington, Zoe Saldana, Sigourney Weaver', 'Budget': '237 million', 'Revenue': '2.79 billion', 'Genre': 'Action, Adventure, Fantasy', 'Summary': 'A paraplegic marine dispatched to the moon Pandora on a unique mission becomes torn between following his orders and protecting the world he feels is his home.'}"},
]

In [15]:
# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.invoke({}).to_messages())

[HumanMessage(content='Matrix', additional_kwargs={}, response_metadata={}), AIMessage(content="{'Director': 'Wachowski', 'Starring': 'Keanu Reeves', 'Budget': '63 million', 'Revenue': '463 million', 'Genre': 'Action, Sci-Fi', 'Summary': 'A computer hacker learns about the true nature of his reality and his role in the war against its controllers.'}", additional_kwargs={}, response_metadata={}), HumanMessage(content='Avatar', additional_kwargs={}, response_metadata={}), AIMessage(content="{'Director': 'James Cameron', 'Starring': 'Sam Worthington, Zoe Saldana, Sigourney Weaver', 'Budget': '237 million', 'Revenue': '2.79 billion', 'Genre': 'Action, Adventure, Fantasy', 'Summary': 'A paraplegic marine dispatched to the moon Pandora on a unique mission becomes torn between following his orders and protecting the world he feels is his home.'}", additional_kwargs={}, response_metadata={})]


In [16]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Give me the information about the movie. in structured format."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [17]:
from langchain_openai import ChatOpenAI


chain = final_prompt | model

from pydantic import BaseModel

class Movie(BaseModel):
    """Always use this tool to structure your response to the user."""
    director: str
    starring: str
    budget: str
    revenue: str
    genre: str
    summary: str

model_with_structured_output = model.with_structured_output(Movie)

structured_chain = final_prompt | model_with_structured_output

structured_chain.invoke({"input": "Gatsby"})

Movie(director='Baz Luhrmann', starring='Leonardo DiCaprio, Carey Mulligan, Tobey Maguire', budget='105 million', revenue='353.6 million', genre='Drama, Romance', summary='A Midwestern war veteran finds himself drawn to the past and lifestyle of his millionaire neighbor.')